In [1]:
import numpy
import numpy as np
from numpy import asarray
from numpy import zeros
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential,Model
from keras.layers import *
from keras.layers import Dense, Flatten, Embedding
from keras.layers import LSTM, Dropout, Activation , Bidirectional,Dense,Flatten,Activation,RepeatVector,Permute,Multiply
from keras.layers.embeddings import Embedding
from keras.layers.core import*
from keras import initializers, regularizers, constraints, Input
from keras.layers.recurrent import LSTM
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras import backend as K
from keras.models import model_from_json
import codecs
import csv
from nltk import word_tokenize
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import emoji
import gensim
import time
import os
import sys 
import json
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

seed = 7
numpy.random.seed(seed)

Using TensorFlow backend.
/home/ajeya/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ajeya/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ajeya/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ajeya/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [2]:
def ReadOpen(filename,Labelfile):
    data = []
    with codecs.open(filename, 'r',encoding="utf-8", errors="replace") as readFile:
        reader = csv.reader(readFile)
        lines = list(reader)
    count = 0
    for i in lines:
        temp = []
        sentence = ' '.join(i)
        for j in word_tokenize(sentence):
            temp.append(j.lower()) 
            count += 1	  
        data.append(temp)
    labels_pd = pd.read_csv(Labelfile,index_col=False)
    labels = numpy.asarray(labels_pd)

    return data[1:],labels, count-1


def extract_emojis(sentence):
    return [word for word in sentence.split() if str(word.encode('unicode-escape'))[2] == '\\' ]


def char_is_emoji(character):
    if character in emoji.UNICODE_EMOJI:
        return True
    else:
        return False

In [3]:
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [4]:
def Preprocess(docs,count):
    # prepare tokenizer
    t = Tokenizer()
    t.fit_on_texts(docs)
    # vocab_size = len(t.word_index) + 1
    # print(vocab_size)
    # integer encode the documents
    encoded_docs = t.texts_to_sequences(docs)
    # pad documents to a max length of 4 words
    # max_length = 4
    padded_docs = pad_sequences(encoded_docs, padding='post')
    l = len(padded_docs[0])
    # load the whole embedding into memory
    embeddings_index = dict()
    f = open('glove.6B.300d.txt',encoding="utf8")
    for line in tqdm(f):
        values = line.split()
        word = values[0]
        coefs = asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Loaded %s word vectors.' % len(embeddings_index))

    e2v = gensim.models.KeyedVectors.load_word2vec_format('emoji2vec.bin', binary=True)
    nf = 0
    # create a weight matrix for words in training docs
    embedding_matrix = zeros((count, 300))
    for word, i in t.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            # print(word)
            new_em = []
            em = extract_emojis(word)
            for ej in em:
                for c in ej:
                    if char_is_emoji(c):
                        new_em.append(c)
            # print(new_em)
            try:
                if new_em:
                        row = []
                        for e in new_em:

                            row.append(e2v[e])
                        embedding_matrix[i] = np.average(np.asarray(row),axis=0).tolist()
                else:
                    embedding_matrix[i] = [0] * 300
            except:
                embedding_matrix[i] = [0] * 300
                nf += 1

    print(str(nf)+" words not found in vocabulary")

    return padded_docs, embedding_matrix,l

In [5]:
def PrepModel(count,embedding_matrix,l,lrate=0.01):
    #model1 = Sequential()
    #model2 = Sequential()

    input = Input(shape = (926,))

    embedded1 = Embedding(count,300, weights=[embedding_matrix],trainable=False)(input)
    embedded2 = Embedding(count,300, weights=[embedding_matrix],trainable=False)(input)

    lstm1 = Bidirectional(LSTM(100,kernel_initializer='he_normal', activation='sigmoid', dropout=0.5,recurrent_dropout=0.5, unroll=False, return_sequences=False))(embedded1)
    lstm2 = Bidirectional(LSTM(100,kernel_initializer='he_normal', activation='sigmoid', dropout=0.5,recurrent_dropout=0.5, unroll=False, return_sequences=True))(embedded2)

    print("here2\n")

    dense = Dense(1, activation='tanh')(lstm2)
    flatten = Flatten()(dense)
    activation_1 = Activation('softmax')(flatten)
    repeat_vector = RepeatVector(100)(activation_1)
    permute = Permute([2,1])(repeat_vector)
    dense_1 = Dense( 100, activation='sigmoid', name='attention_probs')(lstm1)

    model1 = Model(inputs = input,output = dense_1)
    model2 = Model(inputs = input,output = permute)

    mergedOut = Multiply()([model1.output,model2.output])
    mergedOut = Flatten()(mergedOut)
    mergedOut = Dense(1,activation="sigmoid")(mergedOut)
    model = Model(inputs = input,output = mergedOut)
    model.compile(optimizer=Adam(lr=lrate), loss='binary_crossentropy', metrics=['acc',f1,recall,precision])

    print('No of parameter:', model.count_params())
    print(model.summary())
    print(K.eval(model.optimizer.lr))

    return model

In [ ]:
def ReadTest(filename,Labelfile):
    data = []

    with codecs.open(filename, 'r',encoding="utf-8", errors="replace") as readFile:
        reader = csv.reader(readFile)
        lines = list(reader)
    count = 0
    for i in lines:
        temp = []
        sentence = ' '.join(i)
        for j in word_tokenize(sentence):
            temp.append(j.lower()) 
            count += 1

        data.append(temp)

    labels_pd = pd.read_csv(Labelfile,index_col=False)
    labels = numpy.array(labels_pd['Labels'])
    # labels = numpy.asarray(labels_pd)

    t = Tokenizer()
    t.fit_on_texts(data)
    encoded_docs = t.texts_to_sequences(data)
    padded_docs = pad_sequences(encoded_docs, padding='post',maxlen=926)

    return padded_docs,labels



if __name__ == "__main__":

    with open('settings.json') as data_file:
        data = json.load(data_file)

    lrate = data["Model_settings"]["Learning_rate"]
    num_epochs = data["Model_settings"]["Epochs"]

    filename = data["FileNames"]["Training_file"]
    Labelfile = data["FileNames"]["Label_file"]
    # data,labels = ReadFile(filename,Labelfile)


    print('Reading data...')
    data,labels,count = ReadOpen(filename,Labelfile)
    print('Getting Embeddings...')
    padded_docs, embedding_matrix,l = Preprocess(data,count)
    print('Preparing model...')
    model = PrepModel(count,embedding_matrix,l,lrate)
    print('Training...')

    X_train, X_test, y_train, y_test = train_test_split(padded_docs, labels, test_size=0.2, random_state=seed)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)
    # X_test1, y_test1 = ReadTest("Testing_data.csv","Testing_labels.csv")
    # print("length of X_test,y_test")
    # print(len(X_test),len(y_test))


    earlyStopping=keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=1, mode='auto')
    model.fit(X_train, y_train, validation_data=(X_val,y_val), nb_epoch=num_epochs, verbose=1, callbacks=[earlyStopping])
    loss, accuracy,f1_score,recall_score,precision_score = model.evaluate(X_test, y_test, verbose=1)
    
    print('Accuracy: %f' % (accuracy*100))
    print('F1 Score: %f' % (f1_score*100))
    print('Recall: %f' % (recall_score*100))
    print('Precision: %f' % (precision_score*100))

Reading data...
Getting Embeddings...


400000it [01:22, 4871.80it/s]


Loaded 400000 word vectors.
235 words not found in vocabulary
Preparing model...
here2



/home/ajeya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("at...)`
/home/ajeya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pe...)`
/home/ajeya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
No of parameter: 59472302
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 926)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 926, 300)     29358900    input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 926, 200)     320800      embedding_2[0][0]                
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 926, 1)       201         bidire

/home/ajeya/.local/lib/python3.6/site-packages/ipykernel_launcher.py:59: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.



Train on 3500 samples, validate on 876 samples
Epoch 1/5
